In [45]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import random
from sklearn.metrics import roc_auc_score # Ajout de la métrique AUC

# On suppose que DSN.py est dans le même dossier
# On va utiliser DeepMultiBasisNet qui est souvent plus performant que le simple DeepSpectralNet
from DSN import DeepMultiBasisNet
from SBN import DeepMultiBasisBilinearNet

# --- 1. SETUP & SEED ---
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_seed()
device = torch.device("cuda" if torch.cuda.is_available() 
                 else "cpu")
print("Device:", device)

# --- 2. DATA LOADING (Identique à ton code, propre) ---
def load_movielens(path="data/ml-1m/ratings.dat"):
    # Si tu n'as pas le fichier localement, assure-toi que le path est bon
    # Pour l'exemple, on peut créer un dummy dataframe si le fichier n'existe pas
    try:
        df = pd.read_csv(
            path, sep="::", engine="python", 
            names=["user_id", "movie_id", "rating", "timestamp"]
        )
    except FileNotFoundError:
        print("Fichier non trouvé, génération de données synthétiques pour le test...")
        df = pd.DataFrame({
            "user_id": np.random.randint(0, 100, 10000),
            "movie_id": np.random.randint(0, 50, 10000),
            "rating": np.random.randint(1, 6, 10000),
            "timestamp": range(10000)
        })

    df["label"] = (df["rating"] >= 4).astype(int)
    df = df[df["label"] == 1]
    return df

def temporal_split(df):
    df = df.sort_values(["user_id", "timestamp"])
    train, val, test = [], [], []
    for u, g in df.groupby("user_id"):
        if len(g) < 3:
            train.append(g)
        else:
            train.append(g.iloc[:-2])
            val.append(g.iloc[-2:-1])
            test.append(g.iloc[-1:])
    return pd.concat(train), pd.concat(val), pd.concat(test)

def reindex(train_df, val_df, test_df):
    users = pd.concat([train_df, val_df, test_df])["user_id"].unique()
    movies = pd.concat([train_df, val_df, test_df])["movie_id"].unique()
    user_map = {u: i for i, u in enumerate(users)}
    movie_map = {m: i for i, m in enumerate(movies)}
    for df in [train_df, val_df, test_df]:
        df["user_id"] = df["user_id"].map(user_map)
        df["movie_id"] = df["movie_id"].map(movie_map)
    return train_df, val_df, test_df, user_map, movie_map

# --- 3. DATASETS ---
class MovieLensTrainDataset(Dataset):
    def __init__(self, df, num_movies):
        self.users = torch.tensor(df.user_id.values, dtype=torch.long)
        self.movies = torch.tensor(df.movie_id.values, dtype=torch.long)
        self.positives = set(zip(df.user_id, df.movie_id))
        self.num_movies = num_movies

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        u = self.users[idx]
        pos_m = self.movies[idx]
        while True:
            neg_m = random.randint(0, self.num_movies - 1)
            if (u.item(), neg_m) not in self.positives:
                break
        
        # On retourne u, m, label séparément pour faciliter le batching
        # Shape finale attendue par sample : (2,) pour user, (2,) pour item, (2,) pour label
        return {
            "user_id": torch.tensor([u, u]), 
            "movie_id": torch.tensor([pos_m, neg_m]),
            "label": torch.tensor([1.0, 0.0])
        }

class MovieLensEvalDataset(Dataset):
    def __init__(self, df, positives, num_movies, num_neg=99): # 99 negs est standard pour le ranking
        self.samples = []
        for _, row in df.iterrows():
            u, pos_m = row.user_id, row.movie_id
            negs = []
            while len(negs) < num_neg:
                m = random.randint(0, num_movies - 1)
                if (u, m) not in positives:
                    negs.append(m)
            self.samples.append((u, pos_m, negs))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        u, pos_m, negs = self.samples[idx]
        movies = [pos_m] + negs
        labels = [1] + [0] * len(negs)
        return {
            "user_id": torch.tensor([u] * len(movies)),
            "movie_id": torch.tensor(movies),
            "label": torch.tensor(labels, dtype=torch.float32)
        }

# --- 4. PREPARATION DATA ---
print("Chargement des données...")
df = load_movielens() 
train_df, val_df, test_df = temporal_split(df)
train_df, val_df, test_df, user_map, movie_map = reindex(train_df, val_df, test_df)

num_users = len(user_map)
num_movies = len(movie_map)
print(f"Users: {num_users}, Movies: {num_movies}")

train_ds = MovieLensTrainDataset(train_df, num_movies)
val_ds = MovieLensEvalDataset(val_df, set(zip(train_df.user_id, train_df.movie_id)), num_movies, num_neg=19) # 19 negs pour aller plus vite en validation
test_ds = MovieLensEvalDataset(test_df, set(zip(train_df.user_id, train_df.movie_id)), num_movies, num_neg=99)

train_loader = DataLoader(train_ds, batch_size=512, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False) # Batch size 1 car longueur variable de negs possible (sinon padding)
test_loader = DataLoader(test_ds, batch_size=1, shuffle=False)

# --- 5. LE MODÈLE DSN COMPLET (AVEC EMBEDDINGS) ---
class DSNRecModel(nn.Module):
    def __init__(self, num_users, num_movies, embed_dim, layers_dim, ortho_mode='hard'):
        super().__init__()
        
        # A. Embeddings : Indispensables pour convertir les IDs en vecteurs
        self.user_embedding = nn.Embedding(num_users, embed_dim)
        self.movie_embedding = nn.Embedding(num_movies, embed_dim)
        
        # Initialisation correcte des embeddings
        nn.init.xavier_uniform_(self.user_embedding.weight)
        nn.init.xavier_uniform_(self.movie_embedding.weight)

        # B. Backbone DSN
        # L'entrée du DSN sera : Embed_User + Embed_Movie
        in_dim = embed_dim * 2 
        
        # On insère la dimension d'entrée au début de la liste des layers
        full_layers_dim = [in_dim] + layers_dim
        
        # On utilise DeepMultiBasisNet (plus puissant que DeepSpectralNet simple)
        # Tu peux changer pour DeepSpectralNet si tu préfères
        self.backbone = DeepMultiBasisBilinearNet(
            layers_dim=full_layers_dim,
            num_bases=8,             # Hyperparamètre DSN
            ortho_mode=None,
            use_final_linear=False,   # Important pour avoir des logits non bornés
            use_layernorm=True,
            use_residual=False
        )

    def forward(self, user_ids, movie_ids):
        # 1. Lookups
        u_emb = self.user_embedding(user_ids) # [Batch, Embed_Dim]
        m_emb = self.movie_embedding(movie_ids) # [Batch, Embed_Dim]
        
        # 2. Concaténation (Feature Interaction input)
        x = torch.cat([u_emb, m_emb], dim=-1) # [Batch, Embed_Dim * 2]
        
        # 3. DSN
        logits = self.backbone(x) # [Batch, 1]
        
        return logits

# --- 7. BOUCLES D'ENTRAÎNEMENT CORRIGÉES ---

def train_one_epoch(model, loader):
    model.train()
    total_loss = 0.0
    
    for batch in loader:
        # batch['user_id'] shape : [Batch, 2] (pos, neg)
        # On doit tout aplatir pour que le modèle voit une liste de samples [Batch * 2]
        user_ids = batch["user_id"].view(-1).to(device)
        movie_ids = batch["movie_id"].view(-1).to(device)
        labels = batch["label"].view(-1).float().to(device) # Shape [Batch * 2]

        optimizer.zero_grad()
        
        # Forward
        logits = model(user_ids, movie_ids).squeeze(-1) # Output [Batch*2, 1] -> [Batch*2]
        
        # Loss
        loss = criterion(logits, labels)
        
        # Ajout de la loss d'orthogonalité si mode 'soft' (optionnel ici car 'hard')
        # loss += 0.1 * model.backbone.get_total_ortho_loss(torch.nn.functional.mse_loss)

        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * labels.size(0)

    return total_loss / (len(loader.dataset) * 2) # *2 car chaque sample a 1 pos et 1 neg

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    
    for batch in loader:
        # Ici batch_size=1, mais chaque item contient [1, Num_Candidats]
        user_ids = batch["user_id"].view(-1).to(device)
        movie_ids = batch["movie_id"].view(-1).to(device)
        labels = batch["label"].view(-1).float().to(device)
        
        logits = model(user_ids, movie_ids).squeeze(-1)
        probs = torch.sigmoid(logits)
        
        all_preds.append(probs.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

    # Concaténation globale pour calculer l'AUC global ou moyen
    flat_preds = np.concatenate(all_preds)
    flat_labels = np.concatenate(all_labels)
    
    # Calcul des métriques
    auc = roc_auc_score(flat_labels, flat_preds)
    
    # Précision simple (seuil 0.5)
    acc = ((flat_preds > 0.5) == flat_labels).mean()
    
    return {"auc": auc, "accuracy": acc}

Device: cpu
Chargement des données...
Users: 6038, Movies: 3533


In [50]:

# --- 6. CONFIGURATION ---
EMBED_DIM = 64
HIDDEN_LAYERS = [16,8,1] # Le dernier doit être 1 pour la prédiction binaire

model = DSNRecModel(
    num_users=num_users,
    num_movies=num_movies,
    embed_dim=EMBED_DIM,
    layers_dim=HIDDEN_LAYERS,
    ortho_mode=None
).to(device)

print(f"Modèle créé avec {model.backbone.num_parameters + num_users*EMBED_DIM + num_movies*EMBED_DIM} paramètres.")

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-2, weight_decay=1e-4)

Modèle créé avec 899785 paramètres.


In [51]:
# --- 8. RUN ---
EPOCHS = 35 

for epoch in range(EPOCHS):
    train_loss = train_one_epoch(model, train_loader)
    val_metrics = evaluate(model, val_loader)
    
    print(
        f"Epoch {epoch+1:02d} | "
        f"Train Loss: {train_loss:.4f} | "
        f"Val AUC: {val_metrics['auc']:.4f} | "
        f"Val Acc: {val_metrics['accuracy']*100:.2f}%"
    )

Epoch 01 | Train Loss: 0.4558 | Val AUC: 0.8449 | Val Acc: 74.39%
Epoch 02 | Train Loss: 0.4015 | Val AUC: 0.8703 | Val Acc: 79.50%
Epoch 03 | Train Loss: 0.3659 | Val AUC: 0.8853 | Val Acc: 82.33%
Epoch 04 | Train Loss: 0.3461 | Val AUC: 0.8920 | Val Acc: 83.83%
Epoch 05 | Train Loss: 0.3316 | Val AUC: 0.8949 | Val Acc: 84.47%
Epoch 06 | Train Loss: 0.3200 | Val AUC: 0.8971 | Val Acc: 84.15%
Epoch 07 | Train Loss: 0.3089 | Val AUC: 0.8954 | Val Acc: 83.56%
Epoch 08 | Train Loss: 0.2999 | Val AUC: 0.8976 | Val Acc: 82.68%
Epoch 09 | Train Loss: 0.2914 | Val AUC: 0.8983 | Val Acc: 84.35%
Epoch 10 | Train Loss: 0.2831 | Val AUC: 0.8988 | Val Acc: 84.79%
Epoch 11 | Train Loss: 0.2754 | Val AUC: 0.8965 | Val Acc: 86.62%
Epoch 12 | Train Loss: 0.2681 | Val AUC: 0.8995 | Val Acc: 85.71%
Epoch 13 | Train Loss: 0.2619 | Val AUC: 0.8988 | Val Acc: 86.20%
Epoch 14 | Train Loss: 0.2561 | Val AUC: 0.8973 | Val Acc: 86.12%
Epoch 15 | Train Loss: 0.2504 | Val AUC: 0.8976 | Val Acc: 86.07%


KeyboardInterrupt: 

In [22]:
Epoch 01 | Train Loss: 0.4616 | Val AUC: 0.8428 | Val Acc: 73.09%
Epoch 02 | Train Loss: 0.4041 | Val AUC: 0.8705 | Val Acc: 79.45%
Epoch 03 | Train Loss: 0.3694 | Val AUC: 0.8830 | Val Acc: 82.48%
Epoch 04 | Train Loss: 0.3493 | Val AUC: 0.8903 | Val Acc: 81.40%
Epoch 05 | Train Loss: 0.3345 | Val AUC: 0.8939 | Val Acc: 83.13%
Epoch 06 | Train Loss: 0.3229 | Val AUC: 0.8967 | Val Acc: 83.52%
Epoch 07 | Train Loss: 0.3123 | Val AUC: 0.8968 | Val Acc: 83.95%
Epoch 08 | Train Loss: 0.3010 | Val AUC: 0.8970 | Val Acc: 84.02%
Epoch 09 | Train Loss: 0.2929 | Val AUC: 0.8986 | Val Acc: 86.05%
Epoch 10 | Train Loss: 0.2835 | Val AUC: 0.8980 | Val Acc: 85.06%
Epoch 11 | Train Loss: 0.2757 | Val AUC: 0.8987 | Val Acc: 84.57%
Epoch 12 | Train Loss: 0.2689 | Val AUC: 0.8977 | Val Acc: 85.17%
Epoch 13 | Train Loss: 0.2626 | Val AUC: 0.8996 | Val Acc: 86.23%
Epoch 14 | Train Loss: 0.2569 | Val AUC: 0.9010 | Val Acc: 85.48%
Epoch 15 | Train Loss: 0.2510 | Val AUC: 0.8987 | Val Acc: 86.22%
Epoch 16 | Train Loss: 0.2464 | Val AUC: 0.9008 | Val Acc: 86.50%

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (989051002.py, line 1)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import random
from sklearn.metrics import roc_auc_score

# --- 1. SETUP & SEED ---
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_seed()
device = torch.device("mps" if torch.backends.mps.is_available() 
                 else "cuda" if torch.cuda.is_available() 
                 else "cpu")
print("Device:", device)

# --- 2. DATA LOADING (Strictement identique) ---
def load_movielens(path="data/ml-1m/ratings.dat"):
    try:
        df = pd.read_csv(
            path, sep="::", engine="python", 
            names=["user_id", "movie_id", "rating", "timestamp"]
        )
    except FileNotFoundError:
        print("Mode Démo : Génération de données synthétiques...")
        df = pd.DataFrame({
            "user_id": np.random.randint(0, 100, 10000),
            "movie_id": np.random.randint(0, 50, 10000),
            "rating": np.random.randint(1, 6, 10000),
            "timestamp": range(10000)
        })
    df["label"] = (df["rating"] >= 4).astype(int)
    df = df[df["label"] == 1]
    return df

def temporal_split(df):
    df = df.sort_values(["user_id", "timestamp"])
    train, val, test = [], [], []
    for u, g in df.groupby("user_id"):
        if len(g) < 3:
            train.append(g)
        else:
            train.append(g.iloc[:-2])
            val.append(g.iloc[-2:-1])
            test.append(g.iloc[-1:])
    return pd.concat(train), pd.concat(val), pd.concat(test)

def reindex(train_df, val_df, test_df):
    users = pd.concat([train_df, val_df, test_df])["user_id"].unique()
    movies = pd.concat([train_df, val_df, test_df])["movie_id"].unique()
    user_map = {u: i for i, u in enumerate(users)}
    movie_map = {m: i for i, m in enumerate(movies)}
    for df in [train_df, val_df, test_df]:
        df["user_id"] = df["user_id"].map(user_map)
        df["movie_id"] = df["movie_id"].map(movie_map)
    return train_df, val_df, test_df, user_map, movie_map

# --- 3. DATASETS ---
class MovieLensTrainDataset(Dataset):
    def __init__(self, df, num_movies):
        self.users = torch.tensor(df.user_id.values, dtype=torch.long)
        self.movies = torch.tensor(df.movie_id.values, dtype=torch.long)
        self.positives = set(zip(df.user_id, df.movie_id))
        self.num_movies = num_movies

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        u = self.users[idx]
        pos_m = self.movies[idx]
        while True:
            neg_m = random.randint(0, self.num_movies - 1)
            if (u.item(), neg_m) not in self.positives:
                break
        return {
            "user_id": torch.tensor([u, u]), 
            "movie_id": torch.tensor([pos_m, neg_m]),
            "label": torch.tensor([1.0, 0.0])
        }

class MovieLensEvalDataset(Dataset):
    def __init__(self, df, positives, num_movies, num_neg=99):
        self.samples = []
        for _, row in df.iterrows():
            u, pos_m = row.user_id, row.movie_id
            negs = []
            while len(negs) < num_neg:
                m = random.randint(0, num_movies - 1)
                if (u, m) not in positives:
                    negs.append(m)
            self.samples.append((u, pos_m, negs))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        u, pos_m, negs = self.samples[idx]
        movies = [pos_m] + negs
        labels = [1] + [0] * len(negs)
        return {
            "user_id": torch.tensor([u] * len(movies)),
            "movie_id": torch.tensor(movies),
            "label": torch.tensor(labels, dtype=torch.float32)
        }

# --- 4. PREPARATION DATA ---
print("Chargement des données...")
df = load_movielens() 
train_df, val_df, test_df = temporal_split(df)
train_df, val_df, test_df, user_map, movie_map = reindex(train_df, val_df, test_df)

num_users = len(user_map)
num_movies = len(movie_map)

train_ds = MovieLensTrainDataset(train_df, num_movies)
val_ds = MovieLensEvalDataset(val_df, set(zip(train_df.user_id, train_df.movie_id)), num_movies, num_neg=19)
train_loader = DataLoader(train_ds, batch_size=512, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False)

# --- 5. LE MODÈLE MLP (BASELINE) ---
class MLPRecModel(nn.Module):
    def __init__(self, num_users, num_movies, embed_dim, hidden_layers=[64, 32], dropout=0.0):
        super().__init__()
        
        # 1. Embeddings (Mêmes dimensions que le DSN)
        self.user_embedding = nn.Embedding(num_users, embed_dim)
        self.movie_embedding = nn.Embedding(num_movies, embed_dim)
        
        # Initialisation standard (Xavier)
        nn.init.xavier_uniform_(self.user_embedding.weight)
        nn.init.xavier_uniform_(self.movie_embedding.weight)

        # 2. Construction du MLP
        # Input dim = Embed User + Embed Item
        input_dim = embed_dim * 2
        
        layers = []
        for hidden_dim in hidden_layers:
            layers.append(nn.Linear(input_dim, hidden_dim))
            layers.append(nn.ReLU())  # Activation classique ReLU
            if dropout > 0:
                layers.append(nn.Dropout(dropout))
            input_dim = hidden_dim
            
        # Dernière couche de projection vers 1 seul neurone (le score)
        layers.append(nn.Linear(input_dim, 1))
        
        self.mlp = nn.Sequential(*layers)

    def forward(self, user_ids, movie_ids):
        u_emb = self.user_embedding(user_ids)
        m_emb = self.movie_embedding(movie_ids)
        
        # Concaténation
        x = torch.cat([u_emb, m_emb], dim=-1)
        
        # Passage dans le MLP
        logits = self.mlp(x)
        return logits

# --- 6. CONFIGURATION ---
# On garde les mêmes hyperparamètres pour être "fair-play"
EMBED_DIM = 32
# Le DSN avait [64, 32, 1], ici on définit les couches cachées [64, 32] 
# (la dernière couche vers 1 est gérée automatiquement dans la classe MLP)
HIDDEN_DIM = [64, 32] 

model = MLPRecModel(
    num_users=num_users,
    num_movies=num_movies,
    embed_dim=EMBED_DIM,
    hidden_layers=HIDDEN_DIM,
    dropout=0.0
).to(device)

print(f"Modèle MLP créé avec {sum(p.numel() for p in model.parameters())} paramètres.")

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

# --- 7. TRAINING LOOP ---
def train_one_epoch(model, loader):
    model.train()
    total_loss = 0.0
    for batch in loader:
        user_ids = batch["user_id"].view(-1).to(device)
        movie_ids = batch["movie_id"].view(-1).to(device)
        labels = batch["label"].view(-1).float().to(device)

        optimizer.zero_grad()
        logits = model(user_ids, movie_ids).squeeze(-1)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * labels.size(0)
    return total_loss / (len(loader.dataset) * 2)

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    for batch in loader:
        user_ids = batch["user_id"].view(-1).to(device)
        movie_ids = batch["movie_id"].view(-1).to(device)
        labels = batch["label"].view(-1).float().to(device)
        
        logits = model(user_ids, movie_ids).squeeze(-1)
        probs = torch.sigmoid(logits)
        all_preds.append(probs.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

    flat_preds = np.concatenate(all_preds)
    flat_labels = np.concatenate(all_labels)
    return {
        "auc": roc_auc_score(flat_labels, flat_preds),
        "accuracy": ((flat_preds > 0.5) == flat_labels).mean()
    }

Device: mps
Chargement des données...
Modèle MLP créé avec 312545 paramètres.


In [34]:
# --- 8. RUN ---
EPOCHS = 18 # On lance sur 11 époques comme ton précédent run
print("\n--- DÉBUT DE L'ENTRAÎNEMENT MLP ---")
for epoch in range(EPOCHS):
    train_loss = train_one_epoch(model, train_loader)
    val_metrics = evaluate(model, val_loader)
    print(f"Epoch {epoch+1:02d} | Train Loss: {train_loss:.4f} | Val AUC: {val_metrics['auc']:.4f} | Val Acc: {val_metrics['accuracy']*100:.2f}%")


--- DÉBUT DE L'ENTRAÎNEMENT MLP ---
Epoch 01 | Train Loss: 0.3368 | Val AUC: 0.8907 | Val Acc: 82.05%
Epoch 02 | Train Loss: 0.3314 | Val AUC: 0.8917 | Val Acc: 82.49%
Epoch 03 | Train Loss: 0.3258 | Val AUC: 0.8933 | Val Acc: 82.96%
Epoch 04 | Train Loss: 0.3214 | Val AUC: 0.8936 | Val Acc: 83.33%
Epoch 05 | Train Loss: 0.3159 | Val AUC: 0.8946 | Val Acc: 84.12%
Epoch 06 | Train Loss: 0.3115 | Val AUC: 0.8944 | Val Acc: 83.70%


KeyboardInterrupt: 

In [155]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import random
from sklearn.metrics import roc_auc_score

# Import de ton architecture SBN
# Si SBN.py n'est pas trouvé, assure-toi qu'il est dans le répertoire courant
from SBN import SpectralBillinearNet

# --- 1. SETUP & SEED ---
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_seed()
device = torch.device("cuda" if torch.cuda.is_available() 
                 else "cpu")
print("Device:", device)

# --- 2. DATA LOADING (Standardisé) ---
def load_movielens(path="data/ml-1m/ratings.dat"):
    try:
        df = pd.read_csv(
            path, sep="::", engine="python", 
            names=["user_id", "movie_id", "rating", "timestamp"]
        )
    except FileNotFoundError:
        print("Mode Démo : Génération de données synthétiques...")
        df = pd.DataFrame({
            "user_id": np.random.randint(0, 100, 10000),
            "movie_id": np.random.randint(0, 50, 10000),
            "rating": np.random.randint(1, 6, 10000),
            "timestamp": range(10000)
        })
    df["label"] = (df["rating"] >= 4).astype(int)
    df = df[df["label"] == 1]
    return df

def temporal_split(df):
    df = df.sort_values(["user_id", "timestamp"])
    train, val, test = [], [], []
    for u, g in df.groupby("user_id"):
        if len(g) < 3:
            train.append(g)
        else:
            train.append(g.iloc[:-2])
            val.append(g.iloc[-2:-1])
            test.append(g.iloc[-1:])
    return pd.concat(train), pd.concat(val), pd.concat(test)

def reindex(train_df, val_df, test_df):
    users = pd.concat([train_df, val_df, test_df])["user_id"].unique()
    movies = pd.concat([train_df, val_df, test_df])["movie_id"].unique()
    user_map = {u: i for i, u in enumerate(users)}
    movie_map = {m: i for i, m in enumerate(movies)}
    for df in [train_df, val_df, test_df]:
        df["user_id"] = df["user_id"].map(user_map)
        df["movie_id"] = df["movie_id"].map(movie_map)
    return train_df, val_df, test_df, user_map, movie_map

# --- 3. DATASETS ---
class MovieLensTrainDataset(Dataset):
    def __init__(self, df, num_movies):
        self.users = torch.tensor(df.user_id.values, dtype=torch.long)
        self.movies = torch.tensor(df.movie_id.values, dtype=torch.long)
        self.positives = set(zip(df.user_id, df.movie_id))
        self.num_movies = num_movies

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        u = self.users[idx]
        pos_m = self.movies[idx]
        while True:
            neg_m = random.randint(0, self.num_movies - 1)
            if (u.item(), neg_m) not in self.positives:
                break
        return {
            "user_id": torch.tensor([u, u]), 
            "movie_id": torch.tensor([pos_m, neg_m]),
            "label": torch.tensor([1.0, 0.0])
        }

class MovieLensEvalDataset(Dataset):
    def __init__(self, df, positives, num_movies, num_neg=99):
        self.samples = []
        for _, row in df.iterrows():
            u, pos_m = row.user_id, row.movie_id
            negs = []
            while len(negs) < num_neg:
                m = random.randint(0, num_movies - 1)
                if (u, m) not in positives:
                    negs.append(m)
            self.samples.append((u, pos_m, negs))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        u, pos_m, negs = self.samples[idx]
        movies = [pos_m] + negs
        labels = [1] + [0] * len(negs)
        return {
            "user_id": torch.tensor([u] * len(movies)),
            "movie_id": torch.tensor(movies),
            "label": torch.tensor(labels, dtype=torch.float32)
        }

# --- 4. PREPARATION DATA ---
print("Chargement des données...")
df = load_movielens() 
train_df, val_df, test_df = temporal_split(df)
train_df, val_df, test_df, user_map, movie_map = reindex(train_df, val_df, test_df)

num_users = len(user_map)
num_movies = len(movie_map)

train_ds = MovieLensTrainDataset(train_df, num_movies)
val_ds = MovieLensEvalDataset(val_df, set(zip(train_df.user_id, train_df.movie_id)), num_movies, num_neg=19)
train_loader = DataLoader(train_ds, batch_size=512, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False)

# --- 5. LE MODÈLE SBN (SPECTRAL BILINEAR) WRAPPER ---
class SBNRecModel(nn.Module):
    def __init__(self, num_users, num_movies, embed_dim, layers_dim, ortho_mode='hard'):
        super().__init__()
        
        # 1. Embeddings (Identiques aux baselines précédentes)
        self.user_embedding = nn.Embedding(num_users, embed_dim)
        self.movie_embedding = nn.Embedding(num_movies, embed_dim)
        
        nn.init.xavier_uniform_(self.user_embedding.weight)
        nn.init.xavier_uniform_(self.movie_embedding.weight)

        # 2. Backbone SBN
        # Input dim = Concat(User, Movie) -> embed_dim * 2
        input_dim = embed_dim * 2
        
        # On insère la dimension d'entrée au début
        full_layers_dim = [input_dim] + layers_dim
        
        self.backbone = DeepMultiBasisBilinearNet(
            layers_dim=full_layers_dim,
            ortho_mode=None,
            num_bases=8,
            use_final_linear=False, # Pour projeter vers le logit final sans activation bizarre
            bias=True,
            use_layernorm=True,     # SBN bénéficie souvent de LayerNorm
            use_residual = True
        )

    def forward(self, user_ids, movie_ids):
        u_emb = self.user_embedding(user_ids)
        m_emb = self.movie_embedding(movie_ids)
        
        # Concaténation des features
        x = torch.cat([u_emb, m_emb], dim=-1)
        
        # Passage dans le réseau bilinéaire spectral
        logits = self.backbone(x)
        
        return logits

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

# --- 7. TRAINING LOOP ---
def train_one_epoch(model, loader):
    model.train()
    total_loss = 0.0
    for batch in loader:
        user_ids = batch["user_id"].view(-1).to(device)
        movie_ids = batch["movie_id"].view(-1).to(device)
        labels = batch["label"].view(-1).float().to(device)

        optimizer.zero_grad()
        logits = model(user_ids, movie_ids).squeeze(-1)
        loss = criterion(logits, labels)
        
        # Si mode 'soft', ajouter la loss d'orthogonalité
        # loss += 0.1 * model.backbone.get_total_ortho_loss(torch.nn.functional.mse_loss)
        
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * labels.size(0)
    return total_loss / (len(loader.dataset) * 2)

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    for batch in loader:
        user_ids = batch["user_id"].view(-1).to(device)
        movie_ids = batch["movie_id"].view(-1).to(device)
        labels = batch["label"].view(-1).float().to(device)
        
        logits = model(user_ids, movie_ids).squeeze(-1)
        probs = torch.sigmoid(logits)
        all_preds.append(probs.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

    flat_preds = np.concatenate(all_preds)
    flat_labels = np.concatenate(all_labels)
    return {
        "auc": roc_auc_score(flat_labels, flat_preds),
        "accuracy": ((flat_preds > 0.5) == flat_labels).mean()
    }


Device: cpu
Chargement des données...


In [156]:
# --- 6. CONFIGURATION ---
EMBED_DIM = 64
HIDDEN_LAYERS = [128,128,64,16, 1] # Output 1 pour la classification binaire

model = SBNRecModel(
    num_users=num_users,
    num_movies=num_movies,
    embed_dim=EMBED_DIM,
    layers_dim=HIDDEN_LAYERS,
    ortho_mode=None # 'hard', 'cayley', ou 'soft'
).to(device)

print(f"Modèle SBN créé avec {model.backbone.num_parameters + num_users*EMBED_DIM + num_movies*EMBED_DIM} paramètres.")


Modèle SBN créé avec 1813041 paramètres.


In [157]:
# --- 8. RUN ---
EPOCHS = 25 
print("\n--- DÉBUT DE L'ENTRAÎNEMENT SBN ---")
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)#, weight_decay=1e-6)  

for epoch in range(EPOCHS):
    train_loss = train_one_epoch(model, train_loader)
    val_metrics = evaluate(model, val_loader)

    print(
        f"Epoch {epoch+1:02d} | "
        f"Train Loss: {train_loss:.4f} | "
        f"Val AUC: {val_metrics['auc']:.4f} | "
        f"Val Acc: {val_metrics['accuracy']*100:.2f}%"
    )


--- DÉBUT DE L'ENTRAÎNEMENT SBN ---
Epoch 01 | Train Loss: 0.4759 | Val AUC: 0.8335 | Val Acc: 74.99%
Epoch 02 | Train Loss: 0.4326 | Val AUC: 0.8538 | Val Acc: 74.16%
Epoch 03 | Train Loss: 0.3903 | Val AUC: 0.8746 | Val Acc: 80.18%
Epoch 04 | Train Loss: 0.3665 | Val AUC: 0.8823 | Val Acc: 83.82%
Epoch 05 | Train Loss: 0.3508 | Val AUC: 0.8881 | Val Acc: 81.06%
Epoch 06 | Train Loss: 0.3374 | Val AUC: 0.8923 | Val Acc: 83.24%
Epoch 07 | Train Loss: 0.3244 | Val AUC: 0.8940 | Val Acc: 80.98%
Epoch 08 | Train Loss: 0.3136 | Val AUC: 0.8948 | Val Acc: 83.52%
Epoch 09 | Train Loss: 0.3040 | Val AUC: 0.8957 | Val Acc: 84.48%
Epoch 10 | Train Loss: 0.2953 | Val AUC: 0.8966 | Val Acc: 84.09%
Epoch 11 | Train Loss: 0.2871 | Val AUC: 0.8981 | Val Acc: 83.59%
Epoch 12 | Train Loss: 0.2798 | Val AUC: 0.8972 | Val Acc: 82.77%


KeyboardInterrupt: 

In [53]:
def inspect_sbn_concepts(model, user_id, movie_id, layer_idx=0):
    """
    Inspecte l'activation des bases bilinéaires (Concepts) pour une paire User-Movie.
    Affiche la 'Résonance' de chaque base : à quel point le modèle réagit-il ?
    """
    model.eval()
    
    # 1. Prépare les inputs
    u_tensor = torch.tensor([user_id]).to(device)
    m_tensor = torch.tensor([movie_id]).to(device)
    
    # 2. Embeddings & Concat
    u_emb = model.user_embedding(u_tensor)
    m_emb = model.movie_embedding(m_tensor)
    x = torch.cat([u_emb, m_emb], dim=-1) # [1, Dim]
    
    # 3. Récupération de la couche ciblée
    # On cherche la couche SBN correspondante dans le backbone
    sbn_layers = [l for l in model.backbone.layers if hasattr(l, 'num_bases')]
    
    if len(sbn_layers) <= layer_idx:
        print("Index de couche invalide ou pas de couche SBN trouvée.")
        return
        
    layer = sbn_layers[layer_idx]
    
    print(f"--- INSPECTION SBN (Couche {layer_idx}) : User {user_id} & Movie {movie_id} ---")
    
    # Score Final pour contexte
    with torch.no_grad():
        logit = model(u_tensor, m_tensor).item()
        prob = torch.sigmoid(torch.tensor(logit)).item()
    print(f"Score Final du Modèle : {logit:.2f} (Prob: {prob*100:.1f}%)")
    print("-" * 50)
    
    activations = []
    
    with torch.no_grad():
        for i in range(layer.num_bases):
            # A. Projections
            right = layer.right_bases[i](x)
            left = layer.left_bases[i](x)
            
            # B. Énergie d'interaction (Produit scalaire implicite)
            # Une valeur élevée signifie que les vecteurs Right et Left sont alignés
            # C'est la "force" de la réaction de cette base
            interaction = (right * left).mean().item()
            
            activations.append((i, interaction))
            
    # 4. Tri par intensité d'activation (Absolue)
    activations.sort(key=lambda x: abs(x[1]), reverse=True)
    
    print(f"Réaction des {layer.num_bases} bases (Concepts) :")
    for base_idx, energy in activations:
        # Visuel
        bar_len = int(abs(energy) * 50) # Facteur d'échelle pour l'affichage
        if energy > 0:
            bar = "🟩" * bar_len
            sign = "+"
        else:
            bar = "🟥" * bar_len
            sign = "-"
            
        print(f"Base #{base_idx} : {sign}{abs(energy):.4f} | {bar}")

# Test sur la couche 0 (la première couche conceptuelle)
inspect_sbn_concepts(model, user_id=2, movie_id=1193, layer_idx=0)

--- INSPECTION SBN (Couche 0) : User 2 & Movie 1193 ---
Score Final du Modèle : 0.92 (Prob: 71.5%)
--------------------------------------------------
Réaction des 8 bases (Concepts) :
Base #5 : -31.6990 | 🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥🟥

In [54]:
def explain_by_ablation(model, user_id, movie_id, layer_idx=0):
    model.eval()
    
    # 1. Préparer les inputs
    u_tensor = torch.tensor([user_id]).to(device)
    m_tensor = torch.tensor([movie_id]).to(device)
    
    # 2. Score original (Référence)
    with torch.no_grad():
        original_logit = model(u_tensor, m_tensor).item()
        original_prob = torch.sigmoid(torch.tensor(original_logit)).item()
    
    print(f"--- ANALYSE D'IMPACT (Ablation) : User {user_id} & Movie {movie_id} ---")
    print(f"Score Original : {original_logit:.4f} (Prob: {original_prob*100:.1f}%)")
    print("-" * 60)
    
    # 3. Accéder à la couche cible
    sbn_layers = [l for l in model.backbone.layers if hasattr(l, 'num_bases')]
    target_layer = sbn_layers[layer_idx]
    
    impacts = []
    
    # 4. On teste chaque base une par une
    # Pour faire ça proprement sans modifier le code du modèle, on va utiliser un "Hook"
    # ou plus simplement : on fait le forward manuellement jusqu'à la couche, on modifie, et on continue.
    # MAIS pour faire simple et robuste : on va patcher temporairement la méthode forward de la couche.
    
    original_forward = target_layer.forward
    
    try:
        for i in range(target_layer.num_bases):
            
            # On définit un forward modifié qui annule JUSTE la base i
            def ablated_forward(x, base_idx_to_kill=i, original_fwd=original_forward):
                # Recopier la logique interne de la couche SBN MultiBasis
                # C'est un peu intrusif, mais c'est le seul moyen précis sans réécrire tout le forward du modèle
                
                # Note: On suppose ici que c'est une MultiBasisBilinearLayer comme dans ton SBN_Multi.py
                all_interactions = []
                for b in range(target_layer.num_bases):
                    right = target_layer.right_bases[b](x)
                    left = target_layer.left_bases[b](x)
                    interaction = right * left
                    
                    # C'EST ICI LA MAGIE : On tue l'interaction si c'est la base cible
                    if b == base_idx_to_kill:
                        interaction = torch.zeros_like(interaction)
                        
                    all_interactions.append(interaction)
                
                combined = torch.cat(all_interactions, dim=-1)
                return target_layer.eigen_weights(combined)

            # On remplace temporairement la méthode
            target_layer.forward = ablated_forward
            
            # On recalcule le score
            with torch.no_grad():
                new_logit = model(u_tensor, m_tensor).item()
            
            # Impact = Différence (Score Original - Score Ablaté)
            # Si Impact > 0 : La base "ajoutait" du score (C'est un motif POSITIF)
            # Si Impact < 0 : La base "enlevait" du score (C'est un motif NEGATIF)
            impact = original_logit - new_logit
            impacts.append((i, impact))
            
    finally:
        # TRES IMPORTANT : Remettre le forward original quoi qu'il arrive
        target_layer.forward = original_forward

    # 5. Affichage trié
    impacts.sort(key=lambda x: abs(x[1]), reverse=True)
    
    for base_idx, impact in impacts:
        if abs(impact) < 0.001: continue # On ignore le bruit
        
        direction = "AIDANT" if impact > 0 else "PENALISANT"
        bar_char = "🟩" if impact > 0 else "🟥"
        bar = bar_char * int(abs(impact) * 20) # Echelle visuelle
        
        print(f"Base #{base_idx} : {impact:+.4f} ({direction}) {bar}")

# Test
explain_by_ablation(model, user_id=2, movie_id=1193)

--- ANALYSE D'IMPACT (Ablation) : User 2 & Movie 1193 ---
Score Original : 0.9183 (Prob: 71.5%)
------------------------------------------------------------
Base #7 : -0.4520 (PENALISANT) 🟥🟥🟥🟥🟥🟥🟥🟥🟥
Base #3 : +0.4152 (AIDANT) 🟩🟩🟩🟩🟩🟩🟩🟩
Base #5 : +0.3664 (AIDANT) 🟩🟩🟩🟩🟩🟩🟩
Base #0 : -0.2508 (PENALISANT) 🟥🟥🟥🟥🟥
Base #1 : -0.1329 (PENALISANT) 🟥🟥
Base #6 : +0.1101 (AIDANT) 🟩🟩
Base #4 : +0.0755 (AIDANT) 🟩
Base #2 : +0.0177 (AIDANT) 


In [55]:
def profile_base_concept(model, base_idx, movie_map, k=5):
    """
    Affiche les films qui activent le plus (positivement ou négativement) une base donnée.
    Permet de comprendre sémantiquement ce que la base a appris.
    """
    # Inverser la map pour avoir ID -> Titre (si tu as les titres, sinon juste ID)
    # Ici on utilise les IDs bruts
    id_to_movie = {v: k for k, v in movie_map.items()} 
    
    print(f"--- PROFILAGE DU CONCEPT (BASE #{base_idx}) ---")
    
    # On va tester la base sur un échantillon de films (ou tous)
    # Pour aller vite, on prend les 100 premiers films du map
    sample_movies = list(movie_map.values())[:500]
    
    activations = []
    layer = model.backbone.layers[0] # On cible la couche SBN
    
    # On crée un faux batch d'utilisateurs (on s'intéresse à la caractéristique intrinsèque du film)
    # On peut utiliser une moyenne d'utilisateurs ou un utilisateur neutre (embedding nul)
    # Ici on regarde juste la projection "Gauche" du film (sa caractéristique latente)
    
    with torch.no_grad():
        movie_tensor = torch.tensor(sample_movies).to(device)
        m_emb = model.movie_embedding(movie_tensor)
        
        # On projette via la matrice LEFT de la base cible
        # (Dans le SBN, Right=User, Left=Movie ou inversement selon l'interaction)
        # On regarde la norme de la projection pour voir l'intensité
        proj = layer.left_bases[base_idx](torch.cat([torch.zeros_like(m_emb), m_emb], dim=-1))
        
        # On prend la norme comme "force" du signal
        energies = proj.norm(dim=1)
        
        # Top films
        top_indices = torch.topk(energies, k).indices
        
        print("Films qui définissent ce concept :")
        for idx in top_indices:
            movie_id = sample_movies[idx.item()]
            raw_id = id_to_movie.get(movie_id, "Unknown")
            print(f"  - Movie ID {raw_id} (Force: {energies[idx]:.4f})")

# Exemple : Qu'est-ce que la Base #6 (ton moteur principal) ?
profile_base_concept(model, base_idx=6, movie_map=movie_map)

--- PROFILAGE DU CONCEPT (BASE #6) ---
Films qui définissent ce concept :
  - Movie ID 1961 (Force: 171.9574)
  - Movie ID 2858 (Force: 168.3036)
  - Movie ID 1580 (Force: 165.9115)
  - Movie ID 1210 (Force: 164.1475)
  - Movie ID 34 (Force: 159.4895)


In [60]:
from DSN import *
# --- 2. Le Bloc Transformer Spectral (Le LEGO) ---
class SpectralTransformerBlock(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1, ortho_mode=None):
        super().__init__()
        
        # A. Attention Temporelle (Standard)
        self.attention = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, dropout=dropout, batch_first=True)
        self.ln1 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        
        # B. Logique Spectrale (Ton Innovation SBN)
        # Remplace le gros FeedForward (d_model -> 4*d_model -> d_model)
        self.spectral_ffn = MultiBasisSpectralLayer(
            in_features=d_model,
            out_features=d_model,
            num_bases=4,          # Tu peux augmenter à 8 pour plus de puissance
            ortho_mode=ortho_mode
        )
        
        self.ln2 = nn.LayerNorm(d_model)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # x: [Batch, Seq_Len, Dim]
        
        # 1. Self-Attention (Gère l'historique)
        attn_out, _ = self.attention(x, x, x, attn_mask=mask, need_weights=False)
        x = self.ln1(x + self.dropout1(attn_out))
        
        # 2. Spectral Bilinear Network (Gère le raisonnement)
        spectral_out = self.spectral_ffn(x)
        x = self.ln2(x + self.dropout2(spectral_out))
        
        return x

class SASRecSpectral(nn.Module):
    def __init__(self, num_users, num_items, max_len, d_model=64, num_heads=2, num_layers=2):
        super().__init__()
        
        # Embeddings
        self.item_emb = nn.Embedding(num_items, d_model)
        self.pos_emb = nn.Embedding(max_len, d_model)
        self.dropout = nn.Dropout(0.1)
        
        # --- C'EST ICI QUE TU CHANGES TOUT ---
        # Au lieu de nn.TransformerEncoderLayer, tu utilises TA classe
        self.blocks = nn.ModuleList([
            SpectralTransformerBlock(d_model, num_heads, ortho_mode='soft')
            for _ in range(num_layers)
        ])
        
        self.final_norm = nn.LayerNorm(d_model)
        
    def forward(self, item_seq):
        # item_seq: [Batch, Seq_Len]
        seq_len = item_seq.size(1)
        
        # Création du masque causal (pour ne pas voir le futur)
        # mask = ... (code standard PyTorch pour masque triangulaire)
        
        # Embeddings + Positional
        positions = torch.arange(seq_len, device=item_seq.device).unsqueeze(0)
        x = self.item_emb(item_seq) + self.pos_emb(positions)
        x = self.dropout(x)
        
        # Passage dans tes blocs spectraux
        for block in self.blocks:
            # Note: Ajoute le masque d'attention ici si nécessaire
            x = block(x)
            
        x = self.final_norm(x)
        return x # [Batch, Seq_Len, d_model]

In [61]:
from torch.nn.utils.rnn import pad_sequence

# 1. Fonction pour générer les séquences
def get_sequential_data(df, min_len=3):
    """
    Groupe par user et trie par temps pour créer des séquences [Film1, Film2, ...]
    """
    # Tri vital pour le séquentiel
    df = df.sort_values(by=['user_id', 'timestamp'])
    
    user_groups = df.groupby('user_id')
    sequences = []
    
    for user_id, group in user_groups:
        movie_ids = group['movie_id'].tolist()
        
        # On ne garde que si l'utilisateur a vu assez de films
        if len(movie_ids) < min_len:
            continue
            
        # Format : (User, Sequence_Historique, Target_Item)
        # Pour l'entrainement SASRec classique, on prend tout l'historique sauf le dernier comme input
        # et le dernier comme target "positive".
        
        # Train : Tout sauf les 2 derniers
        train_seq = movie_ids[:-2]
        train_target = movie_ids[-2]
        sequences.append((train_seq, train_target))
        
        # (Pour faire simple ici, on ne fait pas le split complexe Val/Test temporel 
        # mais tu peux adapter si tu veux être rigoureux comme avant)

    return sequences

# 2. Le Dataset Séquentiel
class SequentialDataset(Dataset):
    def __init__(self, sequences, num_items, max_len=50):
        self.sequences = sequences
        self.num_items = num_items
        self.max_len = max_len

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq, pos_target = self.sequences[idx]
        
        # Gestion de la longueur (Padding ou Truncate)
        # On prend les 'max_len' derniers films (le passé récent)
        seq = seq[-self.max_len:]
        
        # Négatif Sampling (pour l'entrainement)
        # On choisit un film au hasard que l'user n'a (probablement) pas vu
        while True:
            neg_target = random.randint(0, self.num_items - 1)
            if neg_target != pos_target and neg_target not in seq:
                break
                
        return {
            "seq": torch.tensor(seq, dtype=torch.long),
            "pos": torch.tensor(pos_target, dtype=torch.long),
            "neg": torch.tensor(neg_target, dtype=torch.long)
        }

# 3. Collate Function (Indispensable pour gérer les tailles variables)
def collate_fn(batch):
    # Padding des séquences pour qu'elles aient la même taille dans le batch
    seqs = [item['seq'] for item in batch]
    pos = torch.stack([item['pos'] for item in batch])
    neg = torch.stack([item['neg'] for item in batch])
    
    # Pad avec 0 (ou un ID spécial padding si tu en as un, ici on suppose que 0 est un film valide 
    # donc idéalement faudrait décaler tous les IDs de +1, mais pour le test ça ira)
    seqs_padded = pad_sequence(seqs, batch_first=True, padding_value=0)
    
    return {
        "seq": seqs_padded, 
        "pos": pos, 
        "neg": neg
    }

class SASRecWrapper(nn.Module):
    def __init__(self, sasrec_model):
        super().__init__()
        self.model = sasrec_model
        
    def forward(self, seq, pos_items, neg_items):
        # 1. On passe la séquence dans le Spectral Transformer
        # Output : [Batch, Seq_Len, Dim]
        # On s'intéresse uniquement au DERNIER état de la séquence (le résumé de l'historique)
        log_feats = self.model(seq) 
        final_feat = log_feats[:, -1, :] # [Batch, Dim]
        
        # 2. On récupère les embeddings des items cibles (Pos et Neg)
        # item_emb est dans self.model.item_emb
        pos_emb = self.model.item_emb(pos_items) # [Batch, Dim]
        neg_emb = self.model.item_emb(neg_items) # [Batch, Dim]
        
        # 3. Calcul des scores (Produit Scalaire)
        pos_logits = (final_feat * pos_emb).sum(dim=-1) # Score de compatibilité Positif
        neg_logits = (final_feat * neg_emb).sum(dim=-1) # Score de compatibilité Négatif
        
        return pos_logits, neg_logits

In [69]:
# --- 1. Reconstruction du DF Réindexé Propre ---
# On concatène tes dataframes déjà réindexés pour avoir tout l'historique propre
df_reindexed = pd.concat([train_df, val_df, test_df]).sort_values(['user_id', 'timestamp'])

# IMPORTANT : On décale tous les IDs de +1
# Pourquoi ? Parce que le "Pad Sequence" utilise 0 pour le vide.
# Si un film a l'ID 0, le modèle croira que c'est du vide !
# Donc : 0 = Padding, 1 = Film 0, 2 = Film 1...
df_reindexed['movie_id'] = df_reindexed['movie_id'] + 1
num_items_seq = num_movies + 1 # On agrandit le vocabulaire de 1

print(f"Nouveau vocabulaire items : {num_items_seq} (0 réservé padding)")

# --- 2. Génération des Séquences avec le bon DF ---
# On utilise df_reindexed au lieu de df
seq_data = get_sequential_data(df_reindexed) 

# --- 3. Dataset & Loader ---
# On passe num_items_seq au dataset
train_ds = SequentialDataset(seq_data, num_items_seq, max_len=50)
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True, collate_fn=collate_fn)

# --- 4. Modèle Adapté ---
# Attention : num_items = num_items_seq maintenant
base_model = SASRecSpectral(
    num_users=num_users, 
    num_items=num_items_seq, # Taille augmentée
    max_len=50, 
    d_model=64, 
    num_layers=2
).to(device)

# Padding Index pour l'embedding (Optimisation)
# On dit à PyTorch que l'index 0 est du vide (ne pas apprendre dessus)
base_model.item_emb = nn.Embedding(num_items_seq, 64, padding_idx=0).to(device)

model = SASRecWrapper(base_model).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

import numpy as np
from sklearn.metrics import roc_auc_score

# --- Fonction d'entraînement (1 Epoque) avec Métriques ---
def train_sasrec_epoch_with_metrics(model, loader):
    model.train()
    total_loss = 0
    
    # Listes pour stocker les prédictions de toute l'époque (pour l'AUC global)
    all_pos_scores = []
    all_neg_scores = []
    
    for batch in loader:
        seq = batch['seq'].to(device)
        pos = batch['pos'].to(device)
        neg = batch['neg'].to(device)
        
        optimizer.zero_grad()
        
        # 1. Forward : On récupère les scores (logits)
        pos_logits, neg_logits = model(seq, pos, neg)
        
        # 2. Loss (BCE)
        # On veut que pos_logits soit proche de 1 et neg_logits proche de 0
        pos_labels = torch.ones_like(pos_logits)
        neg_labels = torch.zeros_like(neg_logits)
        
        loss = criterion(pos_logits, pos_labels) + criterion(neg_logits, neg_labels)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # 3. Stockage pour métriques (On détache du graphe pour ne pas saturer la RAM)
        # On applique Sigmoid pour avoir des probabilités entre 0 et 1
        with torch.no_grad():
            pos_probs = torch.sigmoid(pos_logits).cpu().numpy()
            neg_probs = torch.sigmoid(neg_logits).cpu().numpy()
            all_pos_scores.append(pos_probs)
            all_neg_scores.append(neg_probs)
            
    # --- Calcul des Métriques en fin d'époque ---
    # On concatène tout
    all_pos = np.concatenate(all_pos_scores)
    all_neg = np.concatenate(all_neg_scores)
    
    # A. Accuracy (Hit Rate) : Est-ce que Score(Pos) > Score(Neg) ?
    # C'est la métrique la plus intuitive : "Le modèle a-t-il préféré le bon film ?"
    accuracy = (all_pos > all_neg).mean()
    
    # B. AUC : On compare tous les 1 vs tous les 0
    # On crée un grand vecteur [Probs_Pos, Probs_Neg] et un vecteur [1, 1..., 0, 0...]
    y_scores = np.concatenate([all_pos, all_neg])
    y_true = np.concatenate([np.ones_like(all_pos), np.zeros_like(all_neg)])
    
    auc = roc_auc_score(y_true, y_scores)
    
    return total_loss / len(loader), auc, accuracy

# --- Lancement ---

Nouveau vocabulaire items : 3534 (0 réservé padding)


In [78]:
class SASRecWithSpectralHead(nn.Module):
    def __init__(self, sasrec_model, d_model=64, ortho_mode='soft'):
        super().__init__()
        self.backbone = sasrec_model # Ton SASRecSpectral
        
        # --- L'INNOVATION : La Tête Spectrale (Comme ton DSN statique) ---
        # Elle prend [Contexte_User (64) + Item_Cible (64)] -> Sort 1 Score
        self.prediction_head = MultiBasisSpectralLayer(
            in_features=d_model * 2, # Concaténation
            out_features=1,          # Score final
            num_bases=4,
            ortho_mode=ortho_mode
        )
        
    def forward(self, seq, pos_items, neg_items):
        # 1. Extraction du Contexte (Via le Transformer)
        # [Batch, Seq_Len, Dim]
        log_feats = self.backbone(seq) 
        
        # On prend le dernier état (le résumé de l'historique)
        user_context = log_feats[:, -1, :] # [Batch, Dim]
        
        # 2. Récupération des Embeddings Items
        pos_emb = self.backbone.item_emb(pos_items) # [Batch, Dim]
        neg_emb = self.backbone.item_emb(neg_items) # [Batch, Dim]
        
        # 3. INTERACTION SPECTRALE (Au lieu du Dot Product)
        
        # Pour le Positif
        # On concatène le contexte utilisateur et l'item candidat
        pos_input = torch.cat([user_context, pos_emb], dim=-1) # [Batch, Dim*2]
        pos_logits = self.prediction_head(pos_input).squeeze(-1)
        
        # Pour le Négatif
        neg_input = torch.cat([user_context, neg_emb], dim=-1) # [Batch, Dim*2]
        neg_logits = self.prediction_head(neg_input).squeeze(-1)
        
        return pos_logits, neg_logits

# --- MISE EN PLACE ---

# 1. On garde ton modèle de base (le Transformer)
base_model = SASRecSpectral(
    num_users=num_users, 
    num_items=num_items_seq, 
    max_len=50, 
    d_model=64, 
    num_layers=2
).to(device)

# 2. On l'emballe avec la TÊTE SPECTRALE (Le "Cerveau" de décision)
model = SASRecWithSpectralHead(base_model, d_model=64).to(device)

In [85]:
import torch.optim as optim

# 1. Config plus musclée
EPOCHS = 30 # Minimum pour voir la convergence
LR = 1e-3

# 2. Scheduler "OneCycle" (Le secret des Transformers qui convergent vite)
# Il commence doucement, accélère, puis ralentit pour affiner.
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-5) # Moins de decay
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer, 
    max_lr=LR, 
    steps_per_epoch=len(train_loader), 
    epochs=EPOCHS
)

print(f"🚀 Lancement du Turbo Training sur {EPOCHS} époques...")

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    
    for batch in train_loader:
        seq = batch['seq'].to(device)
        pos = batch['pos'].to(device)
        neg = batch['neg'].to(device)
        
        optimizer.zero_grad()
        pos_logits, neg_logits = model(seq, pos, neg)
        
        loss = criterion(pos_logits, torch.ones_like(pos_logits)) + \
               criterion(neg_logits, torch.zeros_like(neg_logits))
        
        loss.backward()
        optimizer.step()
        scheduler.step() # On met à jour le LR à chaque batch
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    
    # Evaluation (une fois par époque)
    if (epoch + 1) % 1 == 0: # On affiche tous les 5 pour pas spammer
        _, auc, acc = train_sasrec_epoch_with_metrics(model, train_loader) # Ou val_loader idéalement
        print(f"Epoch {epoch+1:02d} | Loss: {avg_loss:.4f} | AUC: {auc:.4f} | Acc: {acc*100:.2f}%")
    else:
        print(f"Epoch {epoch+1:02d} | Loss: {avg_loss:.4f}")

🚀 Lancement du Turbo Training sur 30 époques...
Epoch 01 | Loss: 0.2095 | AUC: 0.9904 | Acc: 98.74%
Epoch 02 | Loss: 0.2203 | AUC: 0.9916 | Acc: 98.96%
Epoch 03 | Loss: 0.2229 | AUC: 0.9899 | Acc: 98.79%
Epoch 04 | Loss: 0.2231 | AUC: 0.9898 | Acc: 98.79%
Epoch 05 | Loss: 0.2438 | AUC: 0.9888 | Acc: 98.56%
Epoch 06 | Loss: 0.2492 | AUC: 0.9867 | Acc: 98.36%
Epoch 07 | Loss: 0.2451 | AUC: 0.9880 | Acc: 98.49%
Epoch 08 | Loss: 0.2792 | AUC: 0.9850 | Acc: 98.29%
Epoch 09 | Loss: 0.2835 | AUC: 0.9846 | Acc: 98.36%


KeyboardInterrupt: 

In [ ]:
🚀 Lancement du Turbo Training sur 30 époques...Epoch 01 | Loss: 2.1883Epoch 02 | Loss: 1.8683Epoch 03 | Loss: 1.6445Epoch 04 | Loss: 1.4915Epoch 05 | Loss: 1.3841 | AUC: 0.6765 | Acc: 67.66%Epoch 06 | Loss: 1.2368Epoch 07 | Loss: 1.2088Epoch 08 | Loss: 1.1580Epoch 09 | Loss: 1.1347Epoch 10 | Loss: 1.1020 | AUC: 0.8091 | Acc: 81.39%Epoch 11 | Loss: 1.0144Epoch 12 | Loss: 1.0165Epoch 13 | Loss: 0.9825Epoch 14 | Loss: 0.9499Epoch 15 | Loss: 0.9301 | AUC: 0.8610 | Acc: 86.35%Epoch 16 | Loss: 0.8868Epoch 17 | Loss: 0.8991Epoch 18 | Loss: 0.8595Epoch 19 | Loss: 0.8326Epoch 20 | Loss: 0.8277 | AUC: 0.8923 | Acc: 89.53%Epoch 21 | Loss: 0.7890Epoch 22 | Loss: 0.7886Epoch 23 | Loss: 0.7530Epoch 24 | Loss: 0.7420Epoch 25 | Loss: 0.7393 | AUC: 0.9085 | Acc: 90.94%Epoch 26 | Loss: 0.7313Epoch 27 | Loss: 0.7233Epoch 28 | Loss: 0.7207Epoch 29 | Loss: 0.7095Epoch 30 | Loss: 0.7336 | AUC: 0.9139 | Acc: 91.33%

les 30 d'apres : 🚀 Lancement du Turbo Training sur 30 époques...
Epoch 01 | Loss: 0.7269 | AUC: 0.9122 | Acc: 91.40%Epoch 02 | Loss: 0.7068 | AUC: 0.9177 | Acc: 91.48%Epoch 03 | Loss: 0.7130 | AUC: 0.9151 | Acc: 92.11%Epoch 04 | Loss: 0.7180 | AUC: 0.9136 | Acc: 91.47%Epoch 05 | Loss: 0.7161 | AUC: 0.9119 | Acc: 90.84%Epoch 06 | Loss: 0.7236 | AUC: 0.9094 | Acc: 90.64%Epoch 07 | Loss: 0.7326 | AUC: 0.9041 | Acc: 90.74%Epoch 08 | Loss: 0.7143 | AUC: 0.9107 | Acc: 91.18%Epoch 09 | Loss: 0.7265 | AUC: 0.9154 | Acc: 91.81%Epoch 10 | Loss: 0.6811 | AUC: 0.9190 | Acc: 91.63%Epoch 11 | Loss: 0.6718 | AUC: 0.9240 | Acc: 92.08%Epoch 12 | Loss: 0.6649 | AUC: 0.9328 | Acc: 92.87%Epoch 13 | Loss: 0.6412 | AUC: 0.9319 | Acc: 92.92%Epoch 14 | Loss: 0.6259 | AUC: 0.9331 | Acc: 92.91%Epoch 15 | Loss: 0.6095 | AUC: 0.9395 | Acc: 93.84%Epoch 16 | Loss: 0.5811 | AUC: 0.9420 | Acc: 93.72%Epoch 17 | Loss: 0.5711 | AUC: 0.9482 | Acc: 94.85%Epoch 18 | Loss: 0.5550 | AUC: 0.9467 | Acc: 94.05%Epoch 19 | Loss: 0.5472 | AUC: 0.9508 | Acc: 94.63%Epoch 20 | Loss: 0.5321 | AUC: 0.9555 | Acc: 95.34%Epoch 21 | Loss: 0.5284 | AUC: 0.9561 | Acc: 95.08%Epoch 22 | Loss: 0.4839 | AUC: 0.9572 | Acc: 95.41%Epoch 23 | Loss: 0.4849 | AUC: 0.9633 | Acc: 95.84%Epoch 24 | Loss: 0.4677 | AUC: 0.9590 | Acc: 95.66%Epoch 25 | Loss: 0.4680 | AUC: 0.9650 | Acc: 96.30%Epoch 26 | Loss: 0.4660 | AUC: 0.9646 | Acc: 95.89%Epoch 27 | Loss: 0.4720 | AUC: 0.9657 | Acc: 96.35%Epoch 28 | Loss: 0.4652 | AUC: 0.9630 | Acc: 95.64%Epoch 29 | Loss: 0.4406 | AUC: 0.9623 | Acc: 95.71%Epoch 30 | Loss: 0.4517 | AUC: 0.9630 | Acc: 96.11%

les 30 encore autre :
🚀 Lancement du Turbo Training sur 30 époques...
Epoch 01 | Loss: 0.4526 | AUC: 0.9623 | Acc: 95.71%Epoch 02 | Loss: 0.4538 | AUC: 0.9645 | Acc: 95.71%Epoch 03 | Loss: 0.4668 | AUC: 0.9593 | Acc: 95.43%Epoch 04 | Loss: 0.4511 | AUC: 0.9641 | Acc: 95.86%Epoch 05 | Loss: 0.4859 | AUC: 0.9561 | Acc: 95.41%Epoch 06 | Loss: 0.5080 | AUC: 0.9539 | Acc: 94.76%Epoch 07 | Loss: 0.4996 | AUC: 0.9548 | Acc: 94.88%Epoch 08 | Loss: 0.4895 | AUC: 0.9530 | Acc: 94.95%Epoch 09 | Loss: 0.5143 | AUC: 0.9537 | Acc: 94.98%Epoch 10 | Loss: 0.4835 | AUC: 0.9563 | Acc: 95.10%Epoch 11 | Loss: 0.5041 | AUC: 0.9583 | Acc: 95.36%Epoch 12 | Loss: 0.4982 | AUC: 0.9613 | Acc: 95.69%Epoch 13 | Loss: 0.5004 | AUC: 0.9564 | Acc: 95.08%Epoch 14 | Loss: 0.4842 | AUC: 0.9617 | Acc: 95.68%Epoch 15 | Loss: 0.4721 | AUC: 0.9642 | Acc: 96.17%Epoch 16 | Loss: 0.4564 | AUC: 0.9649 | Acc: 96.09%Epoch 17 | Loss: 0.4398 | AUC: 0.9658 | Acc: 96.02%Epoch 18 | Loss: 0.4399 | AUC: 0.9685 | Acc: 96.64%Epoch 19 | Loss: 0.4422 | AUC: 0.9681 | Acc: 96.06%Epoch 20 | Loss: 0.4144 | AUC: 0.9682 | Acc: 96.52%Epoch 21 | Loss: 0.4029 | AUC: 0.9724 | Acc: 96.84%Epoch 22 | Loss: 0.3810 | AUC: 0.9716 | Acc: 96.79%Epoch 23 | Loss: 0.3750 | AUC: 0.9750 | Acc: 96.93%Epoch 24 | Loss: 0.3814 | AUC: 0.9755 | Acc: 97.33%Epoch 25 | Loss: 0.3674 | AUC: 0.9749 | Acc: 97.00%Epoch 26 | Loss: 0.3663 | AUC: 0.9783 | Acc: 97.60%Epoch 27 | Loss: 0.3552 | AUC: 0.9793 | Acc: 97.46%Epoch 28 | Loss: 0.3419 | AUC: 0.9778 | Acc: 97.23%Epoch 29 | Loss: 0.3450 | AUC: 0.9765 | Acc: 96.85%Epoch 30 | Loss: 0.3480 | AUC: 0.9786 | Acc: 97.46%

les 30 encore encore autre : 
🚀 Lancement du Turbo Training sur 30 époques...Epoch 01 | Loss: 0.3380 | AUC: 0.9787 | Acc: 97.27%Epoch 02 | Loss: 0.3524 | AUC: 0.9786 | Acc: 97.65%Epoch 03 | Loss: 0.3638 | AUC: 0.9752 | Acc: 96.85%Epoch 04 | Loss: 0.3533 | AUC: 0.9753 | Acc: 97.27%Epoch 05 | Loss: 0.3854 | AUC: 0.9725 | Acc: 96.69%Epoch 06 | Loss: 0.3773 | AUC: 0.9709 | Acc: 96.88%Epoch 07 | Loss: 0.3999 | AUC: 0.9665 | Acc: 96.34%Epoch 08 | Loss: 0.4273 | AUC: 0.9688 | Acc: 96.34%Epoch 09 | Loss: 0.4237 | AUC: 0.9709 | Acc: 96.69%Epoch 10 | Loss: 0.4225 | AUC: 0.9678 | Acc: 96.69%Epoch 11 | Loss: 0.4079 | AUC: 0.9666 | Acc: 96.37%Epoch 12 | Loss: 0.4000 | AUC: 0.9693 | Acc: 96.79%Epoch 13 | Loss: 0.4027 | AUC: 0.9677 | Acc: 96.42%Epoch 14 | Loss: 0.3926 | AUC: 0.9729 | Acc: 96.93%Epoch 15 | Loss: 0.4037 | AUC: 0.9732 | Acc: 96.93%Epoch 16 | Loss: 0.3624 | AUC: 0.9725 | Acc: 96.93%Epoch 17 | Loss: 0.3618 | AUC: 0.9763 | Acc: 97.28%Epoch 18 | Loss: 0.3630 | AUC: 0.9759 | Acc: 97.08%Epoch 19 | Loss: 0.3447 | AUC: 0.9770 | Acc: 97.22%Epoch 20 | Loss: 0.3565 | AUC: 0.9787 | Acc: 97.70%Epoch 21 | Loss: 0.3365 | AUC: 0.9801 | Acc: 97.43%Epoch 22 | Loss: 0.3242 | AUC: 0.9807 | Acc: 97.66%Epoch 23 | Loss: 0.3190 | AUC: 0.9822 | Acc: 97.96%Epoch 24 | Loss: 0.3191 | AUC: 0.9805 | Acc: 98.01%Epoch 25 | Loss: 0.2910 | AUC: 0.9833 | Acc: 97.91%Epoch 26 | Loss: 0.2823 | AUC: 0.9848 | Acc: 98.26%Epoch 27 | Loss: 0.2786 | AUC: 0.9847 | Acc: 98.04%Epoch 28 | Loss: 0.2944 | AUC: 0.9838 | Acc: 98.14%Epoch 29 | Loss: 0.3035 | AUC: 0.9834 | Acc: 97.83%Epoch 30 | Loss: 0.2708 | AUC: 0.9847 | Acc: 98.28%

les 30 autres :
🚀 Lancement du Turbo Training sur 30 époques...
Epoch 01 | Loss: 0.3010 | AUC: 0.9842 | Acc: 97.93%Epoch 02 | Loss: 0.2971 | AUC: 0.9845 | Acc: 98.38%Epoch 03 | Loss: 0.2812 | AUC: 0.9839 | Acc: 97.98%Epoch 04 | Loss: 0.3098 | AUC: 0.9821 | Acc: 97.75%Epoch 05 | Loss: 0.3209 | AUC: 0.9803 | Acc: 97.66%Epoch 06 | Loss: 0.3235 | AUC: 0.9793 | Acc: 97.60%Epoch 07 | Loss: 0.3481 | AUC: 0.9778 | Acc: 97.46%Epoch 08 | Loss: 0.3809 | AUC: 0.9746 | Acc: 96.88%Epoch 09 | Loss: 0.3576 | AUC: 0.9778 | Acc: 97.35%Epoch 10 | Loss: 0.3541 | AUC: 0.9767 | Acc: 97.27%Epoch 11 | Loss: 0.3862 | AUC: 0.9775 | Acc: 97.55%Epoch 12 | Loss: 0.3475 | AUC: 0.9772 | Acc: 97.61%Epoch 13 | Loss: 0.3290 | AUC: 0.9806 | Acc: 97.80%Epoch 14 | Loss: 0.3475 | AUC: 0.9809 | Acc: 97.95%Epoch 15 | Loss: 0.3305 | AUC: 0.9797 | Acc: 97.45%Epoch 16 | Loss: 0.3194 | AUC: 0.9807 | Acc: 97.73%Epoch 17 | Loss: 0.3344 | AUC: 0.9826 | Acc: 98.11%Epoch 18 | Loss: 0.3093 | AUC: 0.9842 | Acc: 98.31%Epoch 19 | Loss: 0.2861 | AUC: 0.9843 | Acc: 98.19%Epoch 20 | Loss: 0.2994 | AUC: 0.9845 | Acc: 98.39%Epoch 21 | Loss: 0.2685 | AUC: 0.9857 | Acc: 98.18%Epoch 22 | Loss: 0.2734 | AUC: 0.9857 | Acc: 98.24%Epoch 23 | Loss: 0.2692 | AUC: 0.9870 | Acc: 98.67%Epoch 24 | Loss: 0.2505 | AUC: 0.9880 | Acc: 98.61%Epoch 25 | Loss: 0.2499 | AUC: 0.9874 | Acc: 98.56%Epoch 26 | Loss: 0.2562 | AUC: 0.9872 | Acc: 98.39%Epoch 27 | Loss: 0.2489 | AUC: 0.9883 | Acc: 98.67%Epoch 28 | Loss: 0.2572 | AUC: 0.9864 | Acc: 98.51%Epoch 29 | Loss: 0.2518 | AUC: 0.9874 | Acc: 98.49%Epoch 30 | Loss: 0.2433 | AUC: 0.9872 | Acc: 98.34%

les 30 encore autre 
🚀 Lancement du Turbo Training sur 30 époques...
Epoch 01 | Loss: 0.2505 | AUC: 0.9877 | Acc: 98.49%Epoch 02 | Loss: 0.2356 | AUC: 0.9893 | Acc: 98.48%Epoch 03 | Loss: 0.2474 | AUC: 0.9883 | Acc: 98.67%Epoch 04 | Loss: 0.2593 | AUC: 0.9873 | Acc: 98.56%Epoch 05 | Loss: 0.2717 | AUC: 0.9862 | Acc: 98.33%Epoch 06 | Loss: 0.2830 | AUC: 0.9837 | Acc: 98.18%Epoch 07 | Loss: 0.3008 | AUC: 0.9841 | Acc: 98.33%Epoch 08 | Loss: 0.3088 | AUC: 0.9824 | Acc: 98.16%Epoch 09 | Loss: 0.3150 | AUC: 0.9823 | Acc: 98.31%Epoch 10 | Loss: 0.3397 | AUC: 0.9831 | Acc: 97.90%Epoch 11 | Loss: 0.3442 | AUC: 0.9820 | Acc: 98.09%Epoch 12 | Loss: 0.3263 | AUC: 0.9839 | Acc: 98.19%Epoch 13 | Loss: 0.3109 | AUC: 0.9820 | Acc: 98.00%Epoch 14 | Loss: 0.3066 | AUC: 0.9843 | Acc: 98.28%Epoch 15 | Loss: 0.2996 | AUC: 0.9823 | Acc: 98.01%Epoch 16 | Loss: 0.2941 | AUC: 0.9858 | Acc: 98.51%Epoch 17 | Loss: 0.2691 | AUC: 0.9843 | Acc: 98.06%Epoch 18 | Loss: 0.2702 | AUC: 0.9856 | Acc: 98.33%Epoch 19 | Loss: 0.2582 | AUC: 0.9858 | Acc: 98.43%Epoch 20 | Loss: 0.2696 | AUC: 0.9881 | Acc: 98.51%Epoch 21 | Loss: 0.2451 | AUC: 0.9872 | Acc: 98.49%Epoch 22 | Loss: 0.2335 | AUC: 0.9905 | Acc: 98.77%Epoch 23 | Loss: 0.2336 | AUC: 0.9899 | Acc: 98.82%Epoch 24 | Loss: 0.2404 | AUC: 0.9903 | Acc: 98.76%Epoch 25 | Loss: 0.2279 | AUC: 0.9911 | Acc: 98.89%Epoch 26 | Loss: 0.2132 | AUC: 0.9892 | Acc: 98.81%Epoch 27 | Loss: 0.2302 | AUC: 0.9900 | Acc: 98.87%Epoch 28 | Loss: 0.2249 | AUC: 0.9909 | Acc: 99.11%Epoch 29 | Loss: 0.2136 | AUC: 0.9908 | Acc: 98.87%Epoch 30 | Loss: 0.2143 | AUC: 0.9898 | Acc: 98.71%

In [87]:
def get_train_val_data(df, min_len=3):
    df = df.sort_values(by=['user_id', 'timestamp'])
    user_groups = df.groupby('user_id')
    
    train_seqs = []
    val_seqs = []
    
    for user_id, group in user_groups:
        movie_ids = group['movie_id'].tolist()
        if len(movie_ids) < min_len: continue
            
        # --- TRAIN : Historique jusqu'à avant-avant-dernier -> Cible : Avant-dernier ---
        # Ex: [A, B, C, D, E] -> Input [A, B, C], Target D
        train_seqs.append((movie_ids[:-2], movie_ids[-2]))
        
        # --- VAL : Historique jusqu'à avant-dernier -> Cible : Dernier ---
        # Ex: [A, B, C, D, E] -> Input [A, B, C, D], Target E
        # Attention : L'input de validation contient la cible d'entrainement !
        val_seqs.append((movie_ids[:-1], movie_ids[-1]))

    return train_seqs, val_seqs

# Recréation des datasets
train_data, val_data = get_train_val_data(df_reindexed) # Utilise ton df propre

train_ds = SequentialDataset(train_data, num_items_seq, max_len=50)
val_ds = SequentialDataset(val_data, num_items_seq, max_len=50)

# Loaders
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_ds, batch_size=128, shuffle=False, collate_fn=collate_fn) # Shuffle False pour Val

In [88]:
# Fonction d'évaluation dédiée (Ne fait pas de backprop)
def evaluate_model(model, loader):
    model.eval()
    all_pos, all_neg = [], []
    
    with torch.no_grad():
        for batch in loader:
            seq = batch['seq'].to(device)
            pos = batch['pos'].to(device)
            neg = batch['neg'].to(device)
            
            pos_logits, neg_logits = model(seq, pos, neg)
            
            all_pos.append(torch.sigmoid(pos_logits).cpu().numpy())
            all_neg.append(torch.sigmoid(neg_logits).cpu().numpy())
            
    all_pos = np.concatenate(all_pos)
    all_neg = np.concatenate(all_neg)
    
    acc = (all_pos > all_neg).mean()
    auc = roc_auc_score(
        np.concatenate([np.ones_like(all_pos), np.zeros_like(all_neg)]),
        np.concatenate([all_pos, all_neg])
    )
    return auc, acc

# TEST DE VERITE
real_auc, real_acc = evaluate_model(model, val_loader)
print(f"🌟 SCORE RÉEL (Validation) : AUC = {real_auc:.4f} | Acc = {real_acc*100:.2f}%")

🌟 SCORE RÉEL (Validation) : AUC = 0.7229 | Acc = 73.90%


In [95]:
import torch
import torch.nn as nn

class SASRecSpectral(nn.Module):
    # J'ai ajouté 'dropout' dans les arguments ici vvv
    def __init__(self, num_users, num_items, max_len, d_model=64, num_heads=2, num_layers=2, dropout=0.1):
        super().__init__()
        
        # Embeddings
        # Note : padding_idx=0 est crucial pour ignorer les 0 ajoutés par le pad_sequence
        self.item_emb = nn.Embedding(num_items, d_model, padding_idx=0)
        self.pos_emb = nn.Embedding(max_len, d_model)
        self.dropout = nn.Dropout(dropout) # On utilise le dropout passé en argument
        
        # Blocs Séquentiels
        self.blocks = nn.ModuleList([
            # On passe le dropout à chaque bloc interne
            SpectralTransformerBlock(d_model, num_heads, dropout=dropout, ortho_mode='soft')
            for _ in range(num_layers)
        ])
        
        self.final_norm = nn.LayerNorm(d_model)
        
    def forward(self, item_seq):
        # item_seq : [Batch, Seq_Len]
        seq_len = item_seq.size(1)
        
        # --- Masque Causal (Empêche de voir le futur) ---
        # Crée une matrice triangulaire : 
        # [0, -inf, -inf]
        # [0, 0, -inf]
        # [0, 0, 0]
        # Tout ce qui est -inf sera ignoré par l'attention
        mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool().to(item_seq.device)
        
        # Embeddings + Position
        positions = torch.arange(seq_len, device=item_seq.device).unsqueeze(0)
        
        # Vérification de sécurité pour les positions (si seq_len > max_len)
        if seq_len > self.pos_emb.num_embeddings:
             positions = positions[:, :self.pos_emb.num_embeddings]
             
        x = self.item_emb(item_seq) + self.pos_emb(positions)
        x = self.dropout(x)
        
        # Passage dans les blocs
        for block in self.blocks:
            # On passe le masque à l'attention
            x = block(x, mask=mask)
            
        x = self.final_norm(x)
        return x

In [111]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import roc_auc_score
import copy

# --- HYPERPARAMETRES (Mode Anti-Overfitting) ---
EPOCHS = 50
LR = 1e-3              # Plus lent pour mieux apprendre (avant: 1e-3)
WEIGHT_DECAY = 1e-4    # Pénalité pour empêcher les poids d'exploser (avant: 1e-5)
DROPOUT = 0.1          # On éteint 30% des neurones aléatoirement (avant: 0.1)

# --- RE-INITIALISATION DU MODELE (Pour appliquer le Dropout) ---
# On recrée le modèle pour être sûr d'avoir les bons paramètres
base_model = SASRecSpectral(
    num_users=num_users, 
    num_items=num_items_seq, 
    max_len=50, 
    d_model=128, 
    num_layers=4,
    dropout=DROPOUT # <--- Important !
).to(device)

# Padding Index
base_model.item_emb = nn.Embedding(num_items_seq, 128, padding_idx=0).to(device)

# Le Wrapper avec Tête Spectrale
model = SASRecWithSpectralHead(base_model, d_model=128).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer, 
    max_lr=LR, 
    steps_per_epoch=len(train_loader), 
    epochs=EPOCHS
)

criterion = nn.BCEWithLogitsLoss()

In [112]:
# --- FONCTIONS UTILITAIRES ---

def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    
    for batch in loader:
        seq = batch['seq'].to(device)
        pos = batch['pos'].to(device)
        neg = batch['neg'].to(device)
        
        optimizer.zero_grad()
        pos_logits, neg_logits = model(seq, pos, neg)
        
        # Loss sur Positifs (Target 1) et Négatifs (Target 0)
        loss = criterion(pos_logits, torch.ones_like(pos_logits)) + \
               criterion(neg_logits, torch.zeros_like(neg_logits))
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    return total_loss / len(loader)

def evaluate(model, loader):
    model.eval()
    all_pos, all_neg = [], []
    
    with torch.no_grad():
        for batch in loader:
            seq = batch['seq'].to(device)
            pos = batch['pos'].to(device)
            neg = batch['neg'].to(device)
            
            pos_logits, neg_logits = model(seq, pos, neg)
            
            all_pos.append(torch.sigmoid(pos_logits).cpu().numpy())
            all_neg.append(torch.sigmoid(neg_logits).cpu().numpy())
            
    # Concaténation
    if len(all_pos) == 0: return 0, 0 # Sécurité si loader vide
    
    all_pos = np.concatenate(all_pos)
    all_neg = np.concatenate(all_neg)
    
    # Métriques
    accuracy = (all_pos > all_neg).mean()
    
    y_true = np.concatenate([np.ones_like(all_pos), np.zeros_like(all_neg)])
    y_scores = np.concatenate([all_pos, all_neg])
    auc = roc_auc_score(y_true, y_scores)
    
    return auc, accuracy

# --- BOUCLE PRINCIPALE ---

best_auc = 0.0
best_model_wts = copy.deepcopy(model.state_dict())
patience = 10
no_improve = 0

print(f"🚀 Lancement de l'entraînement (Validation stricte)...")
print("-" * 75)
print(f"{'Epoch':^5} | {'Train Loss':^12} | {'VAL AUC':^10} | {'VAL ACC':^10} | {'Status':^10}")
print("-" * 75)

for epoch in range(EPOCHS):
    # 1. Train
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    
    # 2. Validation (La vérité)
    val_auc, val_acc = evaluate(model, val_loader)
    
    # 3. Checkpoint & Affichage
    is_best = val_auc > best_auc
    status = "🔥 BEST" if is_best else ""
    
    print(f"{epoch+1:02d}/{EPOCHS} | {train_loss:.4f}       | {val_auc:.4f}     | {val_acc*100:.2f}%    | {status}")
    
    if is_best:
        best_auc = val_auc
        best_model_wts = copy.deepcopy(model.state_dict())
        no_improve = 0
        # Sauvegarde disque immédiate
        torch.save(model.state_dict(), "best_sasrec_spectral.pth")
    else:
        no_improve += 1
        
    # Early Stopping
    if no_improve >= patience:
        print(f"\n⏹️ Arrêt précoce : Pas d'amélioration depuis {patience} époques.")
        break

print("-" * 75)
print(f"🏆 Meilleur Résultat Final -> Val AUC: {best_auc:.4f}")

# On recharge les meilleurs poids pour finir
model.load_state_dict(best_model_wts)

🚀 Lancement de l'entraînement (Validation stricte)...
---------------------------------------------------------------------------
Epoch |  Train Loss  |  VAL AUC   |  VAL ACC   |   Status  
---------------------------------------------------------------------------
01/50 | 2.1556       | 0.5491     | 55.41%    | 🔥 BEST
02/50 | 1.7447       | 0.5684     | 57.25%    | 🔥 BEST
03/50 | 1.6216       | 0.5751     | 57.05%    | 🔥 BEST
04/50 | 1.5536       | 0.5932     | 58.97%    | 🔥 BEST
05/50 | 1.5054       | 0.6105     | 60.76%    | 🔥 BEST
06/50 | 1.4687       | 0.6186     | 62.29%    | 🔥 BEST
07/50 | 1.4285       | 0.6233     | 62.58%    | 🔥 BEST
08/50 | 1.4195       | 0.6328     | 63.10%    | 🔥 BEST
09/50 | 1.3576       | 0.6408     | 64.52%    | 🔥 BEST
10/50 | 1.3392       | 0.6548     | 65.48%    | 🔥 BEST
11/50 | 1.3337       | 0.6608     | 66.15%    | 🔥 BEST
12/50 | 1.3086       | 0.6689     | 66.46%    | 🔥 BEST
13/50 | 1.2932       | 0.6749     | 67.59%    | 🔥 BEST
14/50 | 1.2709     

KeyboardInterrupt: 

In [114]:
# --- 1. CONFIGURATION "GOLD STANDARD" ---
EPOCHS = 50
LR = 1e-2              # C'est LA clé. Pas 1e-2.
WEIGHT_DECAY = 1e-5    # Légère régularisation
DROPOUT = 0.2          # Suffisant pour 64 dim

# --- 2. MODELE PLUS LEGER & AGILE ---
# On revient à 2 couches / 64 dim.
# Pourquoi ? Parce que si ça marche pas à 64, ça marchera pas à 128.
base_model = SASRecSpectral(
    num_users=num_users, 
    num_items=num_items_seq, 
    max_len=50, 
    d_model=64,       
    num_layers=2,     
    dropout=DROPOUT 
).to(device)

# Padding Index
base_model.item_emb = nn.Embedding(num_items_seq, 64, padding_idx=0).to(device)

# --- 3. TÊTE SIMPLE (Dot Product) ---
# On utilise le wrapper simple. C'est la méthode éprouvée pour SASRec.
# Ton innovation est DANS le backbone (SpectralTransformerBlock), pas besoin d'en rajouter dans la tête.
model = SASRecWrapper(base_model).to(device)

# Optimiseur
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
criterion = nn.BCEWithLogitsLoss()

# --- 4. LANCEMENT ---
print(f"🚀 Lancement Config 'Sanity Check' (LR={LR}, Dim=64)...")
print("-" * 75)
print(f"{'Epoch':^5} | {'Train Loss':^12} | {'VAL AUC':^10} | {'VAL ACC':^10} | {'Status':^10}")
print("-" * 75)

best_auc = 0.0
patience = 0

for epoch in range(EPOCHS):
    # Train
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    
    # Validation
    val_auc, val_acc = evaluate_model(model, val_loader)
    
    # Logique Best
    is_best = val_auc > best_auc
    status = "🔥 BEST" if is_best else ""
    
    print(f"{epoch+1:02d}/{EPOCHS} | {train_loss:.4f}       | {val_auc:.4f}     | {val_acc*100:.2f}%    | {status}")
    
    if is_best:
        best_auc = val_auc
        patience = 0
        torch.save(model.state_dict(), "best_sasrec_spectral_final.pth")
    else:
        patience += 1
        
    if patience > 10:
        print("Stop: Pas d'amélioration.")
        break

🚀 Lancement Config 'Sanity Check' (LR=0.01, Dim=64)...
---------------------------------------------------------------------------
Epoch |  Train Loss  |  VAL AUC   |  VAL ACC   |   Status  
---------------------------------------------------------------------------
01/50 | 4.4262       | 0.6050     | 60.31%    | 🔥 BEST
02/50 | 1.8460       | 0.6453     | 64.49%    | 🔥 BEST
03/50 | 1.4305       | 0.6993     | 70.01%    | 🔥 BEST
04/50 | 1.2762       | 0.7321     | 73.92%    | 🔥 BEST
05/50 | 1.1617       | 0.7598     | 76.16%    | 🔥 BEST
06/50 | 1.0690       | 0.7665     | 76.57%    | 🔥 BEST
07/50 | 1.0434       | 0.7734     | 77.51%    | 🔥 BEST
08/50 | 0.9770       | 0.7744     | 78.11%    | 🔥 BEST
09/50 | 0.9400       | 0.7770     | 77.32%    | 🔥 BEST
10/50 | 0.9354       | 0.7811     | 77.71%    | 🔥 BEST
11/50 | 0.9122       | 0.7806     | 77.90%    | 
12/50 | 0.9160       | 0.7845     | 78.86%    | 🔥 BEST
13/50 | 0.9269       | 0.7802     | 77.71%    | 
14/50 | 0.9256       | 0.7829 

KeyboardInterrupt: 

In [115]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=WEIGHT_DECAY)
for epoch in range(EPOCHS):
    # Train
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    
    # Validation
    val_auc, val_acc = evaluate_model(model, val_loader)
    
    # Logique Best
    is_best = val_auc > best_auc
    status = "🔥 BEST" if is_best else ""
    
    print(f"{epoch+1:02d}/{EPOCHS} | {train_loss:.4f}       | {val_auc:.4f}     | {val_acc*100:.2f}%    | {status}")
    
    if is_best:
        best_auc = val_auc
        patience = 0
        torch.save(model.state_dict(), "best_sasrec_spectral_final.pth")
    else:
        patience += 1
        
    if patience > 10:
        print("Stop: Pas d'amélioration.")
        break

01/50 | 0.8950       | 0.7880     | 78.31%    | 🔥 BEST
02/50 | 0.8621       | 0.7821     | 78.66%    | 
03/50 | 0.8733       | 0.7883     | 78.72%    | 🔥 BEST
04/50 | 0.8730       | 0.7847     | 78.91%    | 
05/50 | 0.8535       | 0.7813     | 78.21%    | 
06/50 | 0.8680       | 0.7849     | 78.61%    | 
07/50 | 0.8574       | 0.7893     | 78.26%    | 🔥 BEST
08/50 | 0.8483       | 0.7916     | 79.75%    | 🔥 BEST
09/50 | 0.8389       | 0.7900     | 78.99%    | 
10/50 | 0.8580       | 0.7897     | 79.04%    | 
11/50 | 0.8778       | 0.7882     | 78.89%    | 
12/50 | 0.8430       | 0.7867     | 78.53%    | 
13/50 | 0.8510       | 0.7878     | 78.72%    | 
14/50 | 0.8421       | 0.7867     | 78.82%    | 
15/50 | 0.8488       | 0.7868     | 78.44%    | 
16/50 | 0.8372       | 0.7899     | 78.09%    | 
17/50 | 0.8429       | 0.7884     | 78.38%    | 
18/50 | 0.8506       | 0.7913     | 79.12%    | 
19/50 | 0.8525       | 0.7915     | 79.24%    | 
Stop: Pas d'amélioration.
